In [1]:
from data import Data
import numpy as np
import math

# Filepath
DATA_DIR = 'data/'

In [2]:
# Get training and test data in Data object
train_data_array = np.loadtxt(DATA_DIR + 'train.csv', delimiter=',', dtype=str)
train_data = Data(data=train_data_array)

test_data_array = np.loadtxt(DATA_DIR + 'test.csv', delimiter=',', dtype=str)
test_data = Data(data=test_data_array)

In [3]:
# For binary classification get which label is more likely
def get_common_label(data):
    labels = data.get_column('label')
    
    first_label = labels[0]
    other_label = None
    
    num_total = len(labels)
    num_first = 0
    
    for label in labels:
        if label == first_label:
            num_first += 1
        elif other_label is None:
            other_label = label
            
    if (num_first/num_total) >= 0.5:
        return first_label
    else:
        return other_label

# Pass in the data you want the entropy computed for
def entropy(value_data):
    
    entropy = 0
    labels = value_data.get_column('label')
    
    num_edible = len(value_data.get_row_subset('label', 'e'))
    num_total = len(labels)
    
    # Probabilties
    prob_edible = num_edible/num_total
    prob_posionous = (num_total-num_edible)/num_total
    
    probabilities = [prob_edible, prob_posionous]
    
    for i in probabilities:
        if i == 0:
            continue
        entropy += (-i*math.log(i,2))
        # print((-i*math.log(i,2)))

    # print("Entropy: " + str(entropy))
    
    return entropy

# Compute the expected entropy across all values for a certain attribute
def expected_entropy(data, attribute):
    
    # Loop through each value for this attribute
    possible_values = data.get_attribute_possible_vals(attribute)
    
    total_num = len(data)
    expected_entropy = 0
    
    for v in possible_values:
        subset = data.get_row_subset(attribute, v)
        if len(subset) == 0:
            continue
        expected_entropy += (len(subset)/total_num) * entropy(subset)
    
    # print('Expected entropy for ' + attribute + ': ' + str(expected_entropy))
    
    return expected_entropy

def decide_split(data, attributes):
    
    # Calculate entropy over entire set
    label_entropy = entropy(data)
    
    info_gain_list = []
    
    highest_gain = 0
    highest_gain_index = -1
    
    # For each attribute in attributes calculate information gain
    for i,attribute in enumerate(attributes):
        # Skip if attribute is the label
        if attribute == 'label':
            continue
            
        info_gain = label_entropy-expected_entropy(data, attribute)
        info_gain_list.append(info_gain)
        
        # print("Information gain for", attribute, ": ", info_gain)
        
        # If this is the new highest gain then update
        if info_gain > highest_gain:
            highest_gain = info_gain
            highest_gain_index = i
    
    #print("Splitting on", attributes[highest_gain_index])
    
    return attributes[highest_gain_index]

    

In [4]:
# Define node class that contains branches and attributes
class Node:
    
    def __init__(self):
        self.is_label = False
        self.branches = {}
        
        # Default setting for label
        self.label = ''
    
    # Attribute getter/setter
    def set_attribute(self, attribute):
        self.attribute = attribute
        
    def get_attribute(self):
        return self.attribute
        
    # Branch getter/setter
    def add_branch(self, branch):
        self.branches[branch] = None
    
    def get_branches(self):
        return self.branches
    
    # Is Label getter/setter
    def is_label(self):
        return self.is_label
    
    def set_is_label(self, boolean):
        self.is_label = boolean
    
    # Label getter/setter
    def set_label(self, label):
        self.label = label
    
    def get_label(self, label):
        return self.label
    
    # Set child
    def add_child(self, branch, child):
        self.branches[branch] = child
    
    # Get max depth of the tree
    def get_depth(self):
        
        if self.is_label:
            return 1
        
        depths = []
        
        for branch in list(self.branches.values()):
            depths.append(branch.get_depth()+1)
        
        return max(depths)
    
    # Predict one specific example at this node
    def predict_example(self, example, column_index_dict):
        if self.is_label:
            return self.label
        else:
            # Get this attribute value
            value = example[column_index_dict[self.attribute]]

            # TODO: Handle case where a certain value wasn't in training data
            if not value in self.branches:
                return self.branches['other'].predict_example(example, column_index_dict)
            else:
                return self.branches[value].predict_example(example, column_index_dict)
    
    # Predict all examples from a data object
    def predict(self, data_object):
        
        data = data_object.raw_data
        column_index_dict = data_object.column_index_dict
        
        predictions = []
        
        for d in data:
            predictions.append(self.predict_example(d, column_index_dict))
        
        return predictions
    
    def describe(self):
        if self.is_label:
            return self.label
        else:
            return self.attribute

In [5]:
# Decision tree class that can train and predict on new data
class DecisionTree:
    
    def __init__(self):
        self.root = None
        
    # Define ID3 algorithm to use in decision tree
    def ID3(self, S, attributes, label, current_depth):

        # if we are limiting max depth then limit it 
        if self.max_depth != -1:
            # If at max depth then take majority here 
            if current_depth >= self.max_depth:
                # Return majority label here
                leaf = Node()

                leaf.set_label(get_common_label(S))
                leaf.set_is_label(True)

                return leaf
        
        # Check if all values in S have the same label
        same_label = True
        first_label = S.get_column(label)[0]
        for temp_label in S.get_column(label):
            if temp_label != first_label:
                same_label = False
                break

        # If all labels are the same then return single node tree with label
        if same_label:
            leaf = Node()

            leaf.set_label(first_label)
            leaf.set_is_label(True)

            return leaf

        # Otherwise create root node for this tree
        root = Node()

        # Use information gain to find best attribute to split on
        attribute_to_split_on = decide_split(S, attributes)
        
        # Set node attribute value
        root.set_attribute(attribute_to_split_on)

        # Get each possible value for that attribute
        possible_values = S.get_attribute_possible_vals(attribute_to_split_on)

        for branch in possible_values:
            # Add new branch corresponding to this value
            root.add_branch(branch)

            # Get subset of this attribute equal to this value
            subset = S.get_row_subset(attribute_to_split_on, branch)

            # Check if this subset is empty
            if len(subset) == 0:
                # Add leaf node with common value of label is s
                leaf = Node()
                leaf.set_is_label(True)

                # Get max label
                leaf.set_label(get_common_label(S))

                # Add leaf node
                root.add_child(branch, leaf)
            else:
                subset_attributes = attributes.copy()
                subset_attributes.remove(attribute_to_split_on)
                root.add_child(branch, self.ID3(subset, subset_attributes, label, current_depth+1))

        # Add other branch for values not in training data and use majority label
        root.add_branch('other')

        # Add child to use if value isn't in values
        leaf = Node()
        leaf.set_is_label(True)
        leaf.set_label(get_common_label(S))
        root.add_child('other', leaf)     

        # Finally return root node
        return root
        
    def train(self, data, max_depth=-1):
        # Get a set of all measured attributes
        measured_attributes_set = list(train_data.column_index_dict.keys())
        
        # Set max depth
        self.max_depth = max_depth
        
        self.root = self.ID3(data, measured_attributes_set, 'label', 0)
        
    def predict(self, data):
        return self.root.predict(data)
        
    def get_depth(self):
        # Minus one to account for root node
        return self.root.get_depth()-1
    
    def print_tree(self):
        self.root.describe_tree()
        
        

In [6]:
# Create a decision tree 
decision_tree = DecisionTree()

decision_tree.train(train_data)

In [7]:
train_predictions = decision_tree.predict(train_data)

train_actual = train_data.get_column('label')

train_total = len(train_actual)
train_correct = (train_predictions == train_actual).sum()

print('Predicting on train data:')
print('Error:', (train_total-train_correct)/train_total)
print('Accuracy:', train_correct/train_total)
print('Correct Predictions:',train_correct, '/', train_total)

Predicting on train data:
Error: 0.0
Accuracy: 1.0
Correct Predictions: 6530 / 6530


In [8]:
test_predictions = decision_tree.predict(test_data)

test_actual = test_data.get_column('label')

test_total = len(test_actual)
test_correct = (test_predictions == test_actual).sum()

print('Predicting on test data:')
print('Error:', (test_total-test_correct)/test_total)
print('Accuracy:', test_correct/test_total)
print('Correct Predictions:',test_correct, '/', test_total)

Predicting on test data:
Error: 0.0
Accuracy: 1.0
Correct Predictions: 1594 / 1594


In [9]:
# k-fold cross-validation
FOLD_DATA_PATH = 'data/CVfolds_new/'

# Load in folds
folds = []

# Get header row
header_row = train_data_array[0]
# Reshape header row
header_row.shape
header_row = np.expand_dims(header_row, axis=0)

# Skip over header rows and read in folds
for i in range(5):
    folds.append(np.loadtxt(FOLD_DATA_PATH + 'fold%s.csv' % (str(i+1)), delimiter=',', dtype=str, skiprows=1))

# Different depths
depths = [1,2,3,4,5,10,15]

In [10]:
# Loop over different depths
depths_accuracy = []
depths_std_dev = []

for j, depth in enumerate(depths):
    # Loop over different groups and calculate average accuracy and std dev
    folds_accuracy = []

    for i in range(len(folds)):
        accuracy_list = []

        # Remove validation fold for this iteration
        folds_copy = folds.copy()
        del folds_copy[i]

        # Combine other folds
        training_folds = np.concatenate((folds_copy))
        # Add header row
        training_folds = np.concatenate((header_row, training_folds))
        
        # Add header row to validation data
        validation_fold = np.concatenate((header_row, folds[i]))
        
        # Use decision tree
        fold_decision_tree = DecisionTree()
        fold_decision_tree.train(Data(data=training_folds), max_depth=depth)
        
        fold_predictions = fold_decision_tree.predict(Data(data=validation_fold))
        fold_correct = (fold_predictions == Data(data=validation_fold).get_column('label')).sum()
        
        folds_accuracy.append(fold_correct/len(fold_predictions))
        
    # Add depth average and std dev
    depths_accuracy.append(np.average(folds_accuracy))
    depths_std_dev.append(np.std(folds_accuracy))

# Print out average accuracies and std dev
for i in range(len(depths_accuracy)):
    print('Depth', depths[i], 'average accuracy =', depths_accuracy[i], 'std dev =', depths_std_dev[i])

Depth 1 average accuracy = 0.767228177641654 std dev = 0.13453882989269683
Depth 2 average accuracy = 0.9595712098009189 std dev = 0.03868519610283838
Depth 3 average accuracy = 0.9784073506891271 std dev = 0.02287576008314949
Depth 4 average accuracy = 0.9796324655436447 std dev = 0.01749143762967826
Depth 5 average accuracy = 0.9843797856049005 std dev = 0.012827192399002712
Depth 10 average accuracy = 0.9848392036753445 std dev = 0.013327523872609227
Depth 15 average accuracy = 0.9848392036753445 std dev = 0.013327523872609227


In [11]:
# Train the tree with the new max depth chosen from the cross validation
decision_tree = DecisionTree()
decision_tree.train(train_data, max_depth=10)

In [12]:
test_predictions = decision_tree.predict(test_data)

test_actual = test_data.get_column('label')

test_total = len(test_actual)
test_correct = (test_predictions == test_actual).sum()

print('Predicting on test data:')
print('Error:', (test_total-test_correct)/test_total)
print('Accuracy:', test_correct/test_total)
print('Correct Predictions:',test_correct, '/', test_total)

Predicting on test data:
Error: 0.0
Accuracy: 1.0
Correct Predictions: 1594 / 1594
